In [1]:
import dcarte 
import pandas as pd 
import iplantuml
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np 

In [2]:
dcarte.__version__

'0.3.32'

In [9]:
assert dcarte.__version__ == '0.3.32'

In [3]:
dcarte.domains()

,RAW,BASE,LEGACY,PROFILE,SLEEP_STUDY,LOOKUP,BED_HABITS
0,Activity,Bed_Occupancy,Device_Type,Activity_Dailies,Diurnal,Device_Types,Bed_Occupancy
1,Appliances,Doors,Doors,Activity_Weeklies,Nocturnal,Homes,
2,Behavioural,Entryway,Entryway,Light,Withings_Nights,Patients,
3,Blood_Pressure,Habitat,Flags,Physiology_Dailies,Withings_Tidy,,
4,Door,Kitchen,Light,Physiology_Weeklies,,,
5,Encounter,Motion,Motion,Sleep_Dailies,,,
6,Environmental,Motion_New,Observation,Sleep_Weeklies,,,
7,Issue,Physiology,Physiology,Temperature,,,
8,Observation_Notes,Sleep,Temperature,,,,
9,Procedure,Transitions,Wellbeing,,,,


In [20]:
df = dcarte.load('Motion','base')

Finished Loading Motion in:                    0.9 seconds   


In [21]:
df.head()

,patient_id,location_name,start_date
0,2GN1PHeHwRzNYQ7q4Nvg7g,Bathroom,2021-05-14 13:34:46
1,2GN1PHeHwRzNYQ7q4Nvg7g,Kitchen,2021-05-14 13:34:51
2,2GN1PHeHwRzNYQ7q4Nvg7g,Bedroom,2021-05-14 13:34:52
3,2GN1PHeHwRzNYQ7q4Nvg7g,Hallway,2021-05-14 13:34:55
4,2GN1PHeHwRzNYQ7q4Nvg7g,Lounge,2021-05-14 13:34:55


In [23]:
df.groupby(['patient_id','location_name']).count()

start_date
patient_id             location_name            
2GN1PHeHwRzNYQ7q4Nvg7g Bathroom             3443
                       Bed_in               1333
                       Bed_out              1333
                       Bedroom             20382
                       Front door           2323
...                                          ...
YYGGmsEaf2gP4JNN9kMprV Bedroom              8554
                       Front door            963
                       Hallway              3822
                       Kitchen             23454
                       Lounge              13735

[725 rows x 1 columns]

In [22]:
df.groupby(['patient_id','location_name']).count().unstack()

start_date                                    \
location_name           Back door Bathroom  Bed_in Bed_out  Bedroom   
patient_id                                                            
2GN1PHeHwRzNYQ7q4Nvg7g        NaN   3443.0  1333.0  1333.0  20382.0   
2TmRrtnsd3WisLYTmw5id1        NaN    126.0     NaN     NaN    408.0   
2zbyXzYNKPwiPtjaA2L64o     3612.0  18405.0   924.0   924.0  20026.0   
3LXU9Vf5dVsXtnJwxAhaN6      463.0  15532.0     NaN     NaN   3418.0   
3hY7Mp7u9YPo1xMARSxLhc     5277.0  16168.0  1366.0  1366.0  18732.0   
...                           ...      ...     ...     ...      ...   
Xv41AaXiVYBKFAa7SLq8F5      323.0   2515.0     7.0     7.0   6706.0   
XvG4pZADpDNdWsQfKrAt9a     1780.0      NaN     NaN     NaN   8376.0   
YJMET4inKfFcBERma2tQRi      467.0   1396.0  1151.0  1151.0   2969.0   
YQb22UBcPbedJqEiKAqvDU        NaN      9.0     NaN     NaN     12.0   
YYGGmsEaf2gP4JNN9kMprV      238.0  13456.0  1266.0  1266.0   8554.0   

                                                              
location_name          Front door  Hallway  Kitchen   Lounge  
patient_id                                                    
2GN1PHeHwRzNYQ7q4Nvg7g     2323.0   9446.0  18652.0  13426.0  
2TmRrtnsd3WisLYTmw5id1       35.0    568.0    384.0    286.0  
2zbyXzYNKPwiPtjaA2L64o     2395.0  39398.0  56778.0  26644.0  
3LXU9Vf5dVsXtnJwxAhaN6      107.0   2735.0      NaN    641.0  
3hY7Mp7u9YPo1xMARSxLhc      796.0      NaN  42128.0  84555.0  
...                           ...      ...      ...      ...  
Xv41AaXiVYBKFAa7SLq8F5      789.0   5456.0   7205.0  12702.0  
XvG4pZADpDNdWsQfKrAt9a     1388.0  23488.0  13582.0   6092.0  
YJMET4inKfFcBERma2tQRi     1215.0   4810.0   5442.0   3099.0  
YQb22UBcPbedJqEiKAqvDU       39.0    118.0    604.0    779.0  
YYGGmsEaf2gP4JNN9kMprV      963.0   3822.0  23454.0  13735.0  

[89 rows x 9 columns]

In [14]:
df = df.query('location_name not in ["Back door","Front door"]')

In [24]:
from dcarte.utils import mine_pathway
pathway = mine_pathway(df.query('patient_id == "2GN1PHeHwRzNYQ7q4Nvg7g"'),
                  value='location_name',
                  source='Bed_out',
                  sink='Bed_in',
                  min_dur=120,
                  max_dur=1200)

In [ ]:
for pid,subset in df.groupby('patient_id'):
    tmp = mine_pathway(subset,**pathway_kwarg)

In [31]:
from dcarte.utils import timer 
@timer('pathway')
def get_pathway(df,kwargs):
    return mine_pathway(df,**kwargs)

In [ ]:
pathway_kwarg = {"value":'location_name',
                 "source":'Bed_out',
                 "sink":'Bed_in',
                 "min_dur":120,
                 "max_dur":1200 }

In [34]:
data = []
for pid,subset in df.groupby('patient_id'):
    print(pid,end=' - ')
    tmp = get_pathway(subset,pathway_kwarg)
    tmp.index = np.repeat(pid,tmp.shape[0])
    data.append(tmp)

2GN1PHeHwRzNYQ7q4Nvg7g - Finished pathway in:                           4.1 seconds   
2TmRrtnsd3WisLYTmw5id1 - Finished pathway in:                           0.0 seconds   
2zbyXzYNKPwiPtjaA2L64o - Finished pathway in:                           6.3 seconds   
3LXU9Vf5dVsXtnJwxAhaN6 - Finished pathway in:                           0.0 seconds   
3hY7Mp7u9YPo1xMARSxLhc - Finished pathway in:                          16.2 seconds   
4h1dAuzg9rdrhyojwxUS26 - Finished pathway in:                          13.3 seconds   
576646jHRwm4KkF9uzRnaC - Finished pathway in:                          12.0 seconds   
5aQpCSrtPFyympF2G14a6c - Finished pathway in:                           0.0 seconds   
5xSoeEijWNFxswzMEAih71 - Finished pathway in:                           5.7 seconds   
64RLyRh1fEUutdhMnvu17d - Finished pathway in:                           1.7 seconds   
6mMTWCmoha9jC4sbTojGei - Finished pathway in:                           0.0 seconds   
6xrbQMkh12eYMdP7Xi1ggc - Finished pathway i

In [43]:
data_ = pd.concat(data)

In [44]:
data_

,start_date,end_date,source,sink,transition,dur,pathway,patient_id
2,2021-05-29 04:58:00,2021-05-29 05:00:00,Bed_out,Bed_in,Bed_out>Bed_in,120.0,Bed_out>Bedroom>Bed_in,2GN1PHeHwRzNYQ7q4Nvg7g
4,2021-05-29 07:10:00,2021-05-29 07:20:00,Bed_out,Bed_in,Bed_out>Bed_in,600.0,Bed_out>Hallway>Bathroom>Bedroom>Bedroom>Bed_in,2GN1PHeHwRzNYQ7q4Nvg7g
8,2021-05-29 16:47:00,2021-05-29 16:57:00,Bed_out,Bed_in,Bed_out>Bed_in,600.0,Bed_out>Bedroom>Bedroom>Bed_in,2GN1PHeHwRzNYQ7q4Nvg7g
10,2021-05-29 17:41:00,2021-05-29 17:47:00,Bed_out,Bed_in,Bed_out>Bed_in,360.0,Bed_out>Bedroom>Bed_in,2GN1PHeHwRzNYQ7q4Nvg7g
12,2021-05-29 18:24:00,2021-05-29 18:30:00,Bed_out,Bed_in,Bed_out>Bed_in,360.0,Bed_out>Bedroom>Bedroom>Bed_in,2GN1PHeHwRzNYQ7q4Nvg7g
...,...,...,...,...,...,...,...,...
2464,2022-02-20 03:57:00,2022-02-20 03:59:00,Bed_out,Bed_in,Bed_out>Bed_in,120.0,Bed_out>Bed_in,YYGGmsEaf2gP4JNN9kMprV
2474,2022-02-21 01:50:00,2022-02-21 01:52:00,Bed_out,Bed_in,Bed_out>Bed_in,120.0,Bed_out>Bed_in,YYGGmsEaf2gP4JNN9kMprV
2488,2022-02-22 09:58:00,2022-02-22 10:00:00,Bed_out,Bed_in,Bed_out>Bed_in,120.0,Bed_out>Bed_in,YYGGmsEaf2gP4JNN9kMprV
2520,2022-02-27 06:05:00,2022-02-27 06:15:00,Bed_out,Bed_in,Bed_out>Bed_in,600.0,Bed_out>Bedroom>Hallway>Bathroom>Bathroom>Bedr...,YYGGmsEaf2gP4JNN9kMprV


In [45]:
vc = data_.pathway.value_counts().to_frame()

In [46]:
vc[vc.pathway>100]

,pathway
Bed_out>Bed_in,45078
Bed_out>Bedroom>Bedroom>Bed_in,1597
Bed_out>Bedroom>Bed_in,1510
Bed_out>Bedroom>Bathroom>Bathroom>Bedroom>Bed_in,1423
Bed_out>Bathroom>Bathroom>Bedroom>Bed_in,764
Bed_out>Bedroom>Bedroom>Bedroom>Bed_in,647
Bed_out>Bedroom>Hallway>Bathroom>Bathroom>Hallway>Bedroom>Bed_in,529
Bed_out>Bedroom>Bathroom>Bathroom>Bathroom>Bedroom>Bed_in,342
Bed_out>Bedroom>Hallway>Hallway>Bedroom>Bed_in,329
Bed_out>Bathroom>Bathroom>Bed_in,324
